# 시군구법정코드 추출

## 순서
1. 데이터 불러오기
2. 데이터를 새로운 데이터프레임으로 생성하기
3. '행정구역'컬럼을 '시도','군구'로 나누기
4. 특정행 수정하기
5. 데이터 저장하기

In [1]:
import requests
import os
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import folium
from folium.plugins import MarkerCluster
import tqdm # 상태바
from tqdm.notebook import tqdm

pd.set_option('display.max_rows',10000)
pd.set_option('display.max_columns',10000)


## 1. 데이터 불러오기

In [2]:
df=pd.read_csv('C:/workspace/주간프로젝트/법정동코드전체자료.csv',encoding='cp949')
df

,법정동코드\t법정동명\t폐지여부
0,1100000000\t서울특별시\t존재
1,1111000000\t서울특별시 종로구\t존재
2,1111010100\t서울특별시 종로구 청운동\t존재
3,1111010200\t서울특별시 종로구 신교동\t존재
4,1111010300\t서울특별시 종로구 궁정동\t존재
...,...
46335,5013032022\t제주특별자치도 서귀포시 표선면 하천리\t존재
46336,5013032023\t제주특별자치도 서귀포시 표선면 성읍리\t존재
46337,5013032024\t제주특별자치도 서귀포시 표선면 가시리\t존재
46338,5013032025\t제주특별자치도 서귀포시 표선면 세화리\t존재


## 데이터를 새로운 데이터프레임으로 생성하기

### 하나의 컬럼을 여러개의 컬럼으로 나누기 

In [3]:
# 하나의 컬럼을 여러개의 컬럼으로 나누기 
code_str=df['법정동코드\t법정동명\t폐지여부'].str.split('\t')
code_str

0                       [1100000000, 서울특별시, 존재]
1                   [1111000000, 서울특별시 종로구, 존재]
2               [1111010100, 서울특별시 종로구 청운동, 존재]
3               [1111010200, 서울특별시 종로구 신교동, 존재]
4               [1111010300, 서울특별시 종로구 궁정동, 존재]
                          ...                  
46335    [5013032022, 제주특별자치도 서귀포시 표선면 하천리, 존재]
46336    [5013032023, 제주특별자치도 서귀포시 표선면 성읍리, 존재]
46337    [5013032024, 제주특별자치도 서귀포시 표선면 가시리, 존재]
46338    [5013032025, 제주특별자치도 서귀포시 표선면 세화리, 존재]
46339    [5013032026, 제주특별자치도 서귀포시 표선면 토산리, 존재]
Name: 법정동코드\t법정동명\t폐지여부, Length: 46340, dtype: object

In [4]:
# 법정동코드, 행정구역, 존재유무의 컬럼으로 나누기 
code=code_str.str.get(0)
address=code_str.str.get(1)
yorn=code_str.str.get(2)
df['코드'],df['행정구역'],df['존재유무']=(code,address,yorn)
df

,법정동코드\t법정동명\t폐지여부,코드,행정구역,존재유무
0,1100000000\t서울특별시\t존재,1100000000,서울특별시,존재
1,1111000000\t서울특별시 종로구\t존재,1111000000,서울특별시 종로구,존재
2,1111010100\t서울특별시 종로구 청운동\t존재,1111010100,서울특별시 종로구 청운동,존재
3,1111010200\t서울특별시 종로구 신교동\t존재,1111010200,서울특별시 종로구 신교동,존재
4,1111010300\t서울특별시 종로구 궁정동\t존재,1111010300,서울특별시 종로구 궁정동,존재
...,...,...,...,...
46335,5013032022\t제주특별자치도 서귀포시 표선면 하천리\t존재,5013032022,제주특별자치도 서귀포시 표선면 하천리,존재
46336,5013032023\t제주특별자치도 서귀포시 표선면 성읍리\t존재,5013032023,제주특별자치도 서귀포시 표선면 성읍리,존재
46337,5013032024\t제주특별자치도 서귀포시 표선면 가시리\t존재,5013032024,제주특별자치도 서귀포시 표선면 가시리,존재
46338,5013032025\t제주특별자치도 서귀포시 표선면 세화리\t존재,5013032025,제주특별자치도 서귀포시 표선면 세화리,존재


### 불필요한 데이터 삭제하기

In [5]:
# 존재유무가 폐지인 행 삭제하기
df=df.drop(df[df.존재유무=='폐지'].index)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20550 entries, 0 to 46339
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   법정동코드	법정동명	폐지여부  20550 non-null  object
 1   코드               20550 non-null  object
 2   행정구역             20550 non-null  object
 3   존재유무             20550 non-null  object
dtypes: object(4)
memory usage: 802.7+ KB


In [7]:
df[df['행정구역']=='전라북도 전주시 완산구']

,법정동코드\t법정동명\t폐지여부,코드,행정구역,존재유무
22506,4511100000\t전라북도 전주시 완산구\t존재,4511100000,전라북도 전주시 완산구,존재


In [8]:
# 사용하지 않을 열 삭제하기
df=df.drop('법정동코드\t법정동명\t폐지여부',axis=1)
df

,코드,행정구역,존재유무
0,1100000000,서울특별시,존재
1,1111000000,서울특별시 종로구,존재
2,1111010100,서울특별시 종로구 청운동,존재
3,1111010200,서울특별시 종로구 신교동,존재
4,1111010300,서울특별시 종로구 궁정동,존재
...,...,...,...
46335,5013032022,제주특별자치도 서귀포시 표선면 하천리,존재
46336,5013032023,제주특별자치도 서귀포시 표선면 성읍리,존재
46337,5013032024,제주특별자치도 서귀포시 표선면 가시리,존재
46338,5013032025,제주특별자치도 서귀포시 표선면 세화리,존재


In [9]:
# 행정구역 중 시군구의 코드 추출하기
df1=df[df.코드.str[6:]=='0000']
df1

,코드,행정구역,존재유무
0,1100000000,서울특별시,존재
1,1111000000,서울특별시 종로구,존재
94,1114000000,서울특별시 중구,존재
179,1117000000,서울특별시 용산구,존재
229,1120000000,서울특별시 성동구,존재
332,1121500000,서울특별시 광진구,존재
342,1123000000,서울특별시 동대문구,존재
399,1126000000,서울특별시 중랑구,존재
422,1129000000,서울특별시 성북구,존재
503,1130500000,서울특별시 강북구,존재


In [10]:
# 시군구 코드 중 시도 만 있는 코드 삭제하기
df1=df1.drop(df1[df1.코드.str[2:]=='00000000'].index)
df1
df1=df1.reset_index(drop=True)
df1

,코드,행정구역,존재유무
0,1111000000,서울특별시 종로구,존재
1,1114000000,서울특별시 중구,존재
2,1117000000,서울특별시 용산구,존재
3,1120000000,서울특별시 성동구,존재
4,1121500000,서울특별시 광진구,존재
5,1123000000,서울특별시 동대문구,존재
6,1126000000,서울특별시 중랑구,존재
7,1129000000,서울특별시 성북구,존재
8,1130500000,서울특별시 강북구,존재
9,1132000000,서울특별시 도봉구,존재


In [11]:
df1.코드=df1.코드.str[:5]
df1

,코드,행정구역,존재유무
0,11110,서울특별시 종로구,존재
1,11140,서울특별시 중구,존재
2,11170,서울특별시 용산구,존재
3,11200,서울특별시 성동구,존재
4,11215,서울특별시 광진구,존재
5,11230,서울특별시 동대문구,존재
6,11260,서울특별시 중랑구,존재
7,11290,서울특별시 성북구,존재
8,11305,서울특별시 강북구,존재
9,11320,서울특별시 도봉구,존재


In [12]:
address_split=df1.행정구역.str.split(' ')
df1['시도'],df1['군구']=(address_split.str.get(0),address_split.str.get(1))

In [13]:
df1=df1.reset_index(drop=True)
df1

,코드,행정구역,존재유무,시도,군구
0,11110,서울특별시 종로구,존재,서울특별시,종로구
1,11140,서울특별시 중구,존재,서울특별시,중구
2,11170,서울특별시 용산구,존재,서울특별시,용산구
3,11200,서울특별시 성동구,존재,서울특별시,성동구
4,11215,서울특별시 광진구,존재,서울특별시,광진구
5,11230,서울특별시 동대문구,존재,서울특별시,동대문구
6,11260,서울특별시 중랑구,존재,서울특별시,중랑구
7,11290,서울특별시 성북구,존재,서울특별시,성북구
8,11305,서울특별시 강북구,존재,서울특별시,강북구
9,11320,서울특별시 도봉구,존재,서울특별시,도봉구


### 특정행 수정하기 

In [14]:
#군구에 '포항시'로 되어있는 행 추출 후 구까지 기입
df1[df1.군구=='포항시']
df1.iloc[254,4]='포항시 남구'
df1.iloc[255,4]='포항시 북구'


In [15]:
#군구에 '창원시'로 되어있는 행 추출 후 구까지 기입
df1[df1.군구=='창원시']
df1.iloc[286,4]='창원시 의창구'
df1.iloc[287,4]='창원시 성산구'
df1.iloc[288,4]='창원시 마산합포구'
df1.iloc[289,4]='창원시 마산회원구'
df1.iloc[290,4]='창원시 진해구'


In [16]:
#군구에 '청주시'로 되어있는 행 추출 후 구까지 기입
df1[df1.군구=='청주시']
df1.iloc[150,4]='청주시 상당구'
df1.iloc[151,4]='청주시 서원구'
df1.iloc[152,4]='청주시 흥덕구'
df1.iloc[153,4]='청주시 청원구'


In [17]:
#군구에 '천안시'로 되어있는 행 추출 후 구까지 기입
df1[df1.군구=='천안시']
df1.iloc[169,4]='천안시 동남구'
df1.iloc[170,4]='천안시 서북구'


In [18]:
#군구에 '전주시'로 되어있는 행 추출 후 구까지 기입
df1[df1.군구=='전주시']
df1.iloc[194,4]='전주시 완산구'
df1.iloc[195,4]='전주시 덕진구'


In [19]:
# '포항시'만 있는 행 삭제 
df1=df1.drop(df1[df1.군구=='포항시'].index)
df1=df1.drop(df1[df1.군구=='창원시'].index)
df1=df1.drop(df1[df1.군구=='청주시'].index)
df1=df1.drop(df1[df1.군구=='천안시'].index)
df1=df1.drop(df1[df1.군구=='전주시'].index)

In [20]:
df1

,코드,행정구역,존재유무,시도,군구
0,11110,서울특별시 종로구,존재,서울특별시,종로구
1,11140,서울특별시 중구,존재,서울특별시,중구
2,11170,서울특별시 용산구,존재,서울특별시,용산구
3,11200,서울특별시 성동구,존재,서울특별시,성동구
4,11215,서울특별시 광진구,존재,서울특별시,광진구
5,11230,서울특별시 동대문구,존재,서울특별시,동대문구
6,11260,서울특별시 중랑구,존재,서울특별시,중랑구
7,11290,서울특별시 성북구,존재,서울특별시,성북구
8,11305,서울특별시 강북구,존재,서울특별시,강북구
9,11320,서울특별시 도봉구,존재,서울특별시,도봉구


In [21]:
# 인덱스 재설정
df1=df1.reset_index(drop=True)

In [22]:
df1

,코드,행정구역,존재유무,시도,군구
0,11110,서울특별시 종로구,존재,서울특별시,종로구
1,11140,서울특별시 중구,존재,서울특별시,중구
2,11170,서울특별시 용산구,존재,서울특별시,용산구
3,11200,서울특별시 성동구,존재,서울특별시,성동구
4,11215,서울특별시 광진구,존재,서울특별시,광진구
5,11230,서울특별시 동대문구,존재,서울특별시,동대문구
6,11260,서울특별시 중랑구,존재,서울특별시,중랑구
7,11290,서울특별시 성북구,존재,서울특별시,성북구
8,11305,서울특별시 강북구,존재,서울특별시,강북구
9,11320,서울특별시 도봉구,존재,서울특별시,도봉구


In [23]:
# 수정한 행 확인
df1.iloc[150:180,:]

,코드,행정구역,존재유무,시도,군구
150,43112,충청북도 청주시 서원구,존재,충청북도,청주시 서원구
151,43113,충청북도 청주시 흥덕구,존재,충청북도,청주시 흥덕구
152,43114,충청북도 청주시 청원구,존재,충청북도,청주시 청원구
153,43130,충청북도 충주시,존재,충청북도,충주시
154,43130,충청북도 충주시 동량면,존재,충청북도,충주시
155,43150,충청북도 제천시,존재,충청북도,제천시
156,43720,충청북도 보은군,존재,충청북도,보은군
157,43720,충청북도 보은군 산외면,존재,충청북도,보은군
158,43730,충청북도 옥천군,존재,충청북도,옥천군
159,43740,충청북도 영동군,존재,충청북도,영동군


In [24]:
del_index=[]
for i in range(len(df1)):
    if(str(df1.iloc[i,1])[-1]=='면'):
        print(df1.iloc[i,1])
        del_index.append(i)

df1=df1.drop(del_index,axis=0)
df1

인천광역시 강화군 교동면
울산광역시 울주군 삼동면
경기도 파주시 진동면
경기도 안성시 죽산면
경기도 화성시 양감면
경기도 포천시 관인면
경기도 양평군 용문면
강원도 춘천시 남산면
충청북도 충주시 동량면
충청북도 보은군 산외면
충청북도 영동군 심천면
충청북도 괴산군 소수면
충청남도 공주시 신풍면
충청남도 보령시 성주면
충청남도 아산시 도고면
충청남도 서산시 고북면
충청남도 논산시 가야곡면
충청남도 부여군 양화면
충청남도 서천군 비인면
충청남도 예산군 오가면
전라북도 군산시 옥서면
전라북도 익산시 금마면
전라북도 정읍시 감곡면
전라북도 남원시 보절면
전라북도 김제시 성덕면
전라북도 완주군 동상면
전라북도 진안군 주천면
전라북도 임실군 덕치면
전라북도 순창군 구림면
전라북도 고창군 흥덕면
전라북도 부안군 하서면
전라남도 순천시 상사면
전라남도 나주시 산포면
전라남도 담양군 수북면
전라남도 곡성군 오산면
전라남도 고흥군 동강면
전라남도 보성군 웅치면
전라남도 화순군 동복면
전라남도 강진군 마량면
전라남도 해남군 황산면
전라남도 장성군 북하면
전라남도 신안군 안좌면
경상북도 김천시 조마면
경상북도 안동시 임동면
경상북도 영천시 대창면
경상북도 상주시 화동면
경상북도 의성군 구천면
경상북도 예천군 지보면
경상북도 울진군 매화면
경상남도 진주시 대곡면
경상남도 의령군 봉수면
경상남도 창녕군 도천면
경상남도 고성군 회화면
경상남도 하동군 청암면
경상남도 산청군 신등면
경상남도 함양군 병곡면
경상남도 거창군 가조면
경상남도 합천군 적중면


,코드,행정구역,존재유무,시도,군구
0,11110,서울특별시 종로구,존재,서울특별시,종로구
1,11140,서울특별시 중구,존재,서울특별시,중구
2,11170,서울특별시 용산구,존재,서울특별시,용산구
3,11200,서울특별시 성동구,존재,서울특별시,성동구
4,11215,서울특별시 광진구,존재,서울특별시,광진구
5,11230,서울특별시 동대문구,존재,서울특별시,동대문구
6,11260,서울특별시 중랑구,존재,서울특별시,중랑구
7,11290,서울특별시 성북구,존재,서울특별시,성북구
8,11305,서울특별시 강북구,존재,서울특별시,강북구
9,11320,서울특별시 도봉구,존재,서울특별시,도봉구


In [25]:
# 인덱스 재설정
df1=df1.reset_index(drop=True)

In [26]:
len(df1)

256

### 데이터 저장하기 

In [27]:
df1.to_csv('시군구법정동코드.csv')

---